In [ ]:
conda install -c anaconda nltk

In [1]:

import sklearn 
sklearn.__version__

'0.23.2'

In [2]:
# import dependencies
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/skirwai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
# Data preprocessing

In [6]:
news_dataset = pd.read_csv('fake-news/train.csv')

In [7]:
news_dataset.shape

(20800, 5)

In [8]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
# cos we have large dataset to no need to replace 
# null values with anyother values

In [11]:
# replacing null values with empty string
news_dataset = news_dataset.fillna('')

In [12]:
news_dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

## Removing columns which are not required

In [13]:
news_dataset = news_dataset.drop(['id','title'], axis=1)

In [14]:
# merging author name and title
news_dataset['context'] = news_dataset['author'] + ' ' + news_dataset['text']

In [15]:
news_dataset['context']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn Ever get the feeling your life...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss Videos 15 Civilians Killed In ...
4        Howard Portnoy Print \nAn Iranian woman has be...
                               ...                        
20795    Jerome Hudson Rapper T. I. unloaded on black c...
20796    Benjamin Hoffman When the Green Bay Packers lo...
20797    Michael J. de la Merced and Rachel Abrams The ...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799    David Swanson   David Swanson is an author, ac...
Name: context, Length: 20800, dtype: object

In [16]:
news_dataset = news_dataset.drop(['author','text'], axis=1)

In [17]:
news_dataset.head()

,label,context
0,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,0,Daniel J. Flynn Ever get the feeling your life...
2,1,Consortiumnews.com Why the Truth Might Get You...
3,1,Jessica Purkiss Videos 15 Civilians Killed In ...
4,1,Howard Portnoy Print \nAn Iranian woman has be...


In [18]:
# seperating data & label
X = news_dataset.drop(['label'], axis=1)
Y = news_dataset['label']

In [19]:
print(X)
print(Y)

                                                 context
0      Darrell Lucus House Dem Aide: We Didn’t Even S...
1      Daniel J. Flynn Ever get the feeling your life...
2      Consortiumnews.com Why the Truth Might Get You...
3      Jessica Purkiss Videos 15 Civilians Killed In ...
4      Howard Portnoy Print \nAn Iranian woman has be...
...                                                  ...
20795  Jerome Hudson Rapper T. I. unloaded on black c...
20796  Benjamin Hoffman When the Green Bay Packers lo...
20797  Michael J. de la Merced and Rachel Abrams The ...
20798  Alex Ansary NATO, Russia To Hold Parallel Exer...
20799  David Swanson   David Swanson is an author, ac...

[20800 rows x 1 columns]
0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


Stemming - Process of reducing word to it root word means in 
example : 
actor, actress, acting ---> act


In [20]:
post_stem = PorterStemmer()

In [25]:
import re, string
def stem(word):
    stem_word = word.lower()
    stem_word = re.sub('[^a-zA-Z]',' ', stem_word)
    stem_word = re.sub('[%s]' % re.escape(string.punctuation), '', stem_word)
    stem_word = stem_word.split()
    stem_word = [post_stem.stem(word) for word in stem_word 
                 if not word in stopwords.words('english')]
    stem_word = ' '.join(stem_word)
    return stem_word
    

In [26]:
news_dataset['context'] = news_dataset['context'].apply(stem)

In [27]:
print(news_dataset['context'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn ever get feel life circl rounda...
2        consortiumnew com truth might get fire octob t...
3        jessica purkiss video civilian kill singl us a...
4        howard portnoy print iranian woman sentenc six...
                               ...                        
20795    jerom hudson rapper unload black celebr met do...
20796    benjamin hoffman green bay packer lost washing...
20797    michael j de la merc rachel abram maci today g...
20798    alex ansari nato russia hold parallel exercis ...
20799    david swanson david swanson author activist jo...
Name: context, Length: 20800, dtype: object


In [94]:
# seperating data and label
X = news_dataset['context'].values
Y = news_dataset['label'].values

In [95]:
print(X)
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet darrel lucu octob subscrib jason chaffetz stump american fork utah imag courtesi michael jolley avail creativ common licens apolog keith olbermann doubt worst person world week fbi director jame comey accord hous democrat aid look like also know second worst person well turn comey sent infam letter announc fbi look email may relat hillari clinton email server rank democrat relev committe hear comey found via tweet one republican committe chairmen know comey notifi republican chairmen democrat rank member hous intellig judiciari oversight committe agenc review email recent discov order see contain classifi inform long letter went oversight committe chairman jason chaffetz set polit world ablaz tweet fbi dir inform fbi learn exist email appear pertin investig case reopen jason chaffetz jasoninthehous octob cours know case comey actual say review email light unrel case know anthoni weiner sext teenag appar littl thing f

In [96]:
print(X.shape,Y.shape)

(20800,) (20800,)


In [ ]:
# splitting dataset to train & test

In [97]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.2,
                                                    stratify = Y,
                                                    random_state=2)

In [34]:
print(X.shape,X_train.shape, X_test.shape)

(20800, 111254) (16640, 111254) (4160, 111254)


In [98]:
# converting textual data to numeriacl data
vectorization = TfidfVectorizer()

X_train = vectorization.fit_transform(X_train)
X_test = vectorization.transform(X_test)

In [99]:
print(X_train)

  (0, 84190)	0.05101949154736515
  (0, 65616)	0.03432425121212794
  (0, 83089)	0.036457003406927355
  (0, 7794)	0.03399861081338625
  (0, 65463)	0.02577195813827463
  (0, 72236)	0.03565041361918782
  (0, 80968)	0.06738056493426206
  (0, 49666)	0.029615539900247224
  (0, 29229)	0.024188190576648606
  (0, 34910)	0.022257072479147454
  (0, 86319)	0.06340060972759658
  (0, 88014)	0.03944382172168233
  (0, 50018)	0.025390609345857042
  (0, 6689)	0.019472474429395547
  (0, 83221)	0.026868362574180394
  (0, 21251)	0.03769188105273968
  (0, 67935)	0.05626232527928179
  (0, 59834)	0.04401262184027362
  (0, 80585)	0.020496636896993176
  (0, 9119)	0.026193212060992226
  (0, 77117)	0.033316063342128256
  (0, 95530)	0.03789860466509528
  (0, 87833)	0.020093588323854095
  (0, 6141)	0.02389695960859631
  (0, 17675)	0.0370324994031323
  :	:
  (16639, 94783)	0.043309460059540186
  (16639, 67961)	0.026661686205120414
  (16639, 56023)	0.09354652577816418
  (16639, 29551)	0.03849965144935989
  (16639, 494

In [100]:
print(Y.shape,Y_train.shape, Y_test.shape)

(20800,) (16640,) (4160,)


trainning model

In [101]:
lr = LogisticRegression()

In [102]:
lr.fit(X_train, Y_train)

LogisticRegression()

Evaluation

accuracy_score onn testing data

In [103]:
predict_lr = lr.predict(X_test)
accuracy_lr = accuracy_score(predict_lr, Y_test)
print(accuracy_lr)

0.9463942307692308


In [104]:
print(classification_report(Y_test, predict_lr))

              precision    recall  f1-score   support

           0       0.94      0.95      0.95      2077
           1       0.95      0.94      0.95      2083

    accuracy                           0.95      4160
   macro avg       0.95      0.95      0.95      4160
weighted avg       0.95      0.95      0.95      4160



In [105]:
# build predictive system

In [106]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(X_train, Y_train)

DecisionTreeClassifier()

In [107]:
predict_dt = DT.predict(X_test)
accuracy_dt = accuracy_score(predict_dt, Y_test)
print(accuracy_dt)

0.8915865384615385


In [108]:
print(classification_report(Y_test, predict_dt))

              precision    recall  f1-score   support

           0       0.89      0.90      0.89      2077
           1       0.90      0.89      0.89      2083

    accuracy                           0.89      4160
   macro avg       0.89      0.89      0.89      4160
weighted avg       0.89      0.89      0.89      4160



# Gradient Boosting Classifier

In [109]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(X_train, Y_train)

GradientBoostingClassifier(random_state=0)

In [110]:
predict_gbc = GBC.predict(X_test)
accuracy_gbc = accuracy_score(predict_gbc, Y_test)
print(accuracy_gbc)

0.9288461538461539


In [111]:
print(classification_report(Y_test, predict_gbc))

              precision    recall  f1-score   support

           0       0.93      0.92      0.93      2077
           1       0.92      0.94      0.93      2083

    accuracy                           0.93      4160
   macro avg       0.93      0.93      0.93      4160
weighted avg       0.93      0.93      0.93      4160



# Random Forest Classifier

In [112]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state=0)
RFC.fit(X_train, Y_train)

RandomForestClassifier(random_state=0)

In [113]:
predict_rfc = RFC.predict(X_test)
accuracy_rfc = accuracy_score(predict_rfc, Y_test)
print(accuracy_rfc)

0.9177884615384615


In [114]:
print(classification_report(Y_test, predict_rfc))

              precision    recall  f1-score   support

           0       0.89      0.95      0.92      2077
           1       0.95      0.88      0.92      2083

    accuracy                           0.92      4160
   macro avg       0.92      0.92      0.92      4160
weighted avg       0.92      0.92      0.92      4160



# Model Testing

In [115]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

In [130]:
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(stem) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = lr.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(
        output_lable(pred_LR[0]), output_lable(pred_DT[0]), output_lable(pred_GBC[0]),
        output_lable(pred_RFC[0])))

In [117]:
X_news = X_test[5]

prediction = lr.predict(X_news)
print(prediction)

if prediction[0] == 0:
    print('news is real')
else:
    print('news is fake')

[1]
news is fake


In [118]:
print(Y_test[5])

1


In [119]:
test_dataset = pd.read_csv('fake-news/test.csv')

In [120]:
test_dataset['context'] = test_dataset['author'] + ' ' + test_dataset['text']

In [121]:
test_dataset = test_dataset.drop(['id','title'], axis=1)

In [122]:
test_dataset = test_dataset.drop(['author','text'], axis=1)

In [123]:
test_dataset.head()

,context
0,"David Streitfeld PALO ALTO, Calif. — After ..."
1,NaN
2,Common Dreams Videos #NoDAPL: Native American ...
3,"Daniel Victor If at first you don’t succeed, t..."
4,Truth Broadcast Network 42 mins ago 1 Views 0 ...


In [124]:
test_dataset = test_dataset.fillna('')

In [125]:
test_dataset.head()

,context
0,"David Streitfeld PALO ALTO, Calif. — After ..."
1,
2,Common Dreams Videos #NoDAPL: Native American ...
3,"Daniel Victor If at first you don’t succeed, t..."
4,Truth Broadcast Network 42 mins ago 1 Views 0 ...


In [126]:
test_dataset.head()

,context
0,"David Streitfeld PALO ALTO, Calif. — After ..."
1,
2,Common Dreams Videos #NoDAPL: Native American ...
3,"Daniel Victor If at first you don’t succeed, t..."
4,Truth Broadcast Network 42 mins ago 1 Views 0 ...


In [131]:
news = str(test_dataset['context'][0])
manual_testing(news)



LR Prediction: Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: Fake News
